In [5]:
import pandas as pd
import re
import spacy
from collections import Counter
import unicodedata
from bs4 import BeautifulSoup
from boilerpipe.extract import Extractor
from sklearn.metrics import classification_report
from nltk.corpus import stopwords
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from nltk.corpus import stopwords
stopwords = set(stopwords.words())
INPUT_FILE = "../1-Data/2-extract-text/extracted_texts/Intel.csv"

nlp = spacy.load('en_core_web_sm')
alternative_company_names = {"AMD (Advanced Micro Devices)": "AMD",
                    'Royal Dutch Shell PLC': "Shell",
                    "Samsung Electronics Co., Ltd.": "Samsung",
                    "Goodyear Tire & Rubber Co": "Goodyear",
                    "Sumitomo Rubber Industries": "Sumitomo",
                    "Exxon Mobil Corp.": "ExxonMobil",
                    "General Motors Corp.": "GM",
                    "Ford Motor Co.": "Ford",
                    "Toyota Motor Corp.": "Toyota",
                    "Petro China": "PetroChina",
                    'Volkswagen AG': "VW"}

def clean_text(html):
    soup = BeautifulSoup(html, "html.parser") # create a new bs4 object from the html data loaded
    for script in soup(["script", "style"]): # remove all javascript and stylesheet code
        script.extract()
    texts = soup.findAll(text=True)
#     import ipdb; ipdb.set_trace()
#     print(len(texts))
    text = ". ".join(t.strip() for t in texts)
    text = unicodedata.normalize("NFKD", text)
    return text

Clean text from html tags

In [6]:
df = pd.read_csv(INPUT_FILE)

In [7]:
len(df)

21

In [8]:
df = pd.read_csv(INPUT_FILE)
df['text'] = df.apply(lambda row: "{} {}".format(row['title'], clean_text(str(row['content']))), axis=1)
df.drop(df[df.text.str.len() < 150].index, inplace=True)

In [9]:
len(df)

21

In [10]:
# COMPANY_NAMES_STOP_WORDS = "PLC|Corp"
with open("company-suffix.txt", "r") as fl:
    text = [i for i in fl.read().split('\n') if not i.startswith('//')]
    COMPANY_NAMES_STOP_WORDS = "(" + "$)|(".join(text) + "$)"

In [15]:
def get_company_names(company):
    company_names = [re.sub(COMPANY_NAMES_STOP_WORDS, '', company, flags=re.IGNORECASE).strip().lower()]
    if company in alternative_company_names:
        company_names.append(alternative_company_names[company].lower())
    #Company is often mentioned by part of it's name. e.g. "Royal Dutch Shell" -> "Shell"
#     company_names = set([company] + [i for i in company.split() if len(i)>2])
    return company_names

# BoW Again  

taken from https://gist.github.com/bbengfort/044682e76def583a12e6c09209c664a1

In [16]:
import os
import time
import string
import pickle

from operator import itemgetter

from nltk.corpus import stopwords as sw
from nltk.corpus import wordnet as wn
from nltk import wordpunct_tokenize
from nltk import WordNetLemmatizer
from nltk import sent_tokenize
from nltk import pos_tag

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import SGDClassifier
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import classification_report as clsr
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cross_validation import train_test_split as tts


def timeit(func):
    """
    Simple timing decorator
    """
    def wrapper(*args, **kwargs):
        start  = time.time()
        result = func(*args, **kwargs)
        delta  = time.time() - start
        return result, delta
    return wrapper


def identity(arg):
    """
    Simple identity function works as a passthrough.
    """
    return arg


class NLTKPreprocessor(BaseEstimator, TransformerMixin):
    """
    Transforms input data by using NLTK tokenization, lemmatization, and
    other normalization and filtering techniques.
    """

    def __init__(self, stopwords=None, punct=None, lower=True, strip=True):
        """
        Instantiates the preprocessor, which make load corpora, models, or do
        other time-intenstive NLTK data loading.
        """
        self.lower      = lower
        self.strip      = strip
        self.stopwords  = set(stopwords) if stopwords else set(sw.words('english'))
        self.punct      = set(punct) if punct else set(string.punctuation)
        self.punct.add("“")
        self.lemmatizer = WordNetLemmatizer()

    def fit(self, X, y=None):
        """
        Fit simply returns self, no other information is needed.
        """
        return self

    def inverse_transform(self, X):
        """
        No inverse transformation
        """
        return X

    def transform(self, X):
        """
        Actually runs the preprocessing on each document.
        """
        return [
            list(self.tokenize(doc)) for doc in X
        ]

    def tokenize(self, document):
        """
        Returns a normalized, lemmatized list of tokens from a document by
        applying segmentation (breaking into sentences), then word/punctuation
        tokenization, and finally part of speech tagging. It uses the part of
        speech tags to look up the lemma in WordNet, and returns the lowercase
        version of all the words, removing stopwords and punctuation.
        """
        # Break the document into sentences
        for sent in sent_tokenize(document):
            # Break the sentence into part of speech tagged tokens
            for token, tag in pos_tag(wordpunct_tokenize(sent)):
                # Apply preprocessing to the token
                token = token.lower() if self.lower else token
                token = token.strip() if self.strip else token
                token = token.strip('_') if self.strip else token
                token = token.strip('*') if self.strip else token

                # If punctuation or stopword, ignore token and continue
                if token in self.stopwords or all(char in self.punct for char in token):
                    continue
                if tag == "NNP":
                    continue
                # Lemmatize the token and yield
                lemma = self.lemmatize(token, tag)
                yield lemma

    def lemmatize(self, token, tag):
        """
        Converts the Penn Treebank tag to a WordNet POS tag, then uses that
        tag to perform much more accurate WordNet lemmatization.
        """
        tag = {
            'N': wn.NOUN,
            'V': wn.VERB,
            'R': wn.ADV,
            'J': wn.ADJ
        }.get(tag[0], wn.NOUN)

        return self.lemmatizer.lemmatize(token, tag)



@timeit
def build_and_evaluate(X, y, classifier=SGDClassifier, outpath=None, verbose=True):
    """
    Builds a classifer for the given list of documents and targets in two
    stages: the first does a train/test split and prints a classifier report,
    the second rebuilds the model on the entire corpus and returns it for
    operationalization.
    X: a list or iterable of raw strings, each representing a document.
    y: a list or iterable of labels, which will be label encoded.
    Can specify the classifier to build with: if a class is specified then
    this will build the model with the Scikit-Learn defaults, if an instance
    is given, then it will be used directly in the build pipeline.
    If outpath is given, this function will write the model as a pickle.
    If verbose, this function will print out information to the command line.
    """

    @timeit
    def build(classifier, X, y=None):
        """
        Inner build function that builds a single model.
        """
        if isinstance(classifier, type):
            classifier = classifier(loss='log')

        model = Pipeline([
            ('preprocessor', NLTKPreprocessor()),
            ('vectorizer', TfidfVectorizer(tokenizer=identity, preprocessor=None, lowercase=False)),
            ('classifier', classifier),
        ])

        model.fit(X, y)
        return model

    # Label encode the targets
    labels = LabelEncoder()
    y = labels.fit_transform(y)

    # Begin evaluation
    if verbose: print("Building for evaluation")
    X_train, X_test, y_train, y_test = tts(X, y, test_size=0.2)
    model, secs = build(classifier, X_train, y_train)

    if verbose: print("Evaluation model fit in {:0.3f} seconds".format(secs))
    if verbose: print("Classification Report:\n")

    y_pred = model.predict(X_test)
    print(clsr(y_test, y_pred, target_names=labels.classes_))

    if verbose: print("Building complete model and saving ...")
    model, secs = build(classifier, X, y)
    model.labels_ = labels

    if verbose: print("Complete model fit in {:0.3f} seconds".format(secs))

    if outpath:
        with open(outpath, 'wb') as f:
            pickle.dump(model, f)

        print("Model written out to {}".format(outpath))

    return model


def show_most_informative_features(model, text=None, n=20):
    """
    Accepts a Pipeline with a classifer and a TfidfVectorizer and computes
    the n most informative features of the model. If text is given, then will
    compute the most informative features for classifying that text.
    Note that this function will only work on linear models with coefs_
    """
    # Extract the vectorizer and the classifier from the pipeline
    vectorizer = model.named_steps['vectorizer']
    classifier = model.named_steps['classifier']

    # Check to make sure that we can perform this computation
    if not hasattr(classifier, 'coef_'):
        raise TypeError(
            "Cannot compute most informative features on {} model.".format(
                classifier.__class__.__name__
            )
        )

    if text is not None:
        # Compute the coefficients for the text
        tvec = model.transform([text]).toarray()
    else:
        # Otherwise simply use the coefficients
        tvec = classifier.coef_

    # Zip the feature names with the coefs and sort
    coefs = sorted(
        zip(tvec[0], vectorizer.get_feature_names()),
        key=itemgetter(0), reverse=False
    )

    topn  = zip(coefs[:n], coefs[:-(n+1):-1])

    # Create the output string to return
    output = []

    # If text, add the predicted value to the output.
    if text is not None:
        output.append("\"{}\"".format(text))
        output.append("Classified as: {}".format(model.predict([text])))
        output.append("")

    # Create two columns with most negative and most positive features.
    for (cp, fnp), (cn, fnn) in topn:
        output.append(
            "{:0.4f}{: >15}    {:0.4f}{: >15}".format(cp, fnp, cn, fnn)
        )

    return "\n".join(output)


In [17]:
import pickle
with open("model.pickle", "rb") as fl:
    model = pickle.load(fl)
df.columns
df['text'] = df.apply(lambda row: "{} {}".format(row['title'], clean_text(str(row['content']))), axis=1)
df.drop(df[df.text.str.len() < 150].index, inplace=True)

In [18]:
len(df)

21

In [19]:
companies = df.groupby("company", as_index=False)
rows_list = []
for num, cp in enumerate(list(companies.groups.keys())):
    neg_url = ''
    pos_url = ''
    cp_df = df.loc[df['company'] == cp]
#     cp_df['lemmatized'] = cp_df.apply(lambda row: " ".join(tokenize(row['text'], get_company_names(row[""]))), axis=1)
    urls = list(cp_df['url'])
#     cp_df['text'] = cp_df.apply(lambda row: "{} {}".format(row[''], clean_text(str(row['content']))), axis=1)
    
    res = model.predict(cp_df['text'])
    counter = Counter(res)
    probs = model.predict_proba(cp_df['text'])
    
    negs, pos, neut = zip(*probs)
    neg_url = urls[negs.index(max(negs))]
    pos_url = urls[pos.index(max(pos))]

    rows_list.append({"company": cp, "pos_c":Counter(res)[1], "neg_c": Counter(res)[0], 'pos_p': sum(pos), 'neg_p': sum(negs), 'neg_url': f"<a href='{neg_url}' target='_blank'>Clickme</a>", 'pos_url': f"<a href='{pos_url}' target='_blank'>Clickme</a>"})#, "neg_url": neg_url,"pos_url": pos_url })
    print(f"{cp}: p: {Counter(res)[1]}, n: {Counter(res)[0]},  'pos_p': {sum(pos)}, 'neg_p': {sum(negs)}, neg_url: {neg_url},  pos_url: {pos_url},")
    
final_df =  pd.DataFrame(rows_list) 

Intel: p: 15, n: 5,  'pos_p': 13.201965950358762, 'neg_p': 4.899903099740847, neg_url: https://mashable.com/2017/12/18/trump-national-security-strategy-omits-climate-change-threat/,  pos_url: https://www.greenbiz.com/article/alphabet-intel-and-walmart-low-carbon-products-make-business-sense,


In [22]:
categories = {"Food": ["Cargill Inc", "Nestle SA", "Procter & Gamble, Co.", "PepsiCo Inc.", "Bunge"],
            "Chemicals": ["BASF AG", "Johnson & Johnson", "Bayer AG", "Ineos", "Dow Chemical"],
            "Electronics": ["Samsung Electronics Co., Ltd.", "Sony Corp.", "AMD (Advanced Micro Devices)", "Toshiba Corp.", "Motorola Inc."],
            "Metals & mining": ["Glencore International AG", "ArcelorMittal", "Agnico-Eagle Mines Ltd.", "ThyssenKrupp Stahl", "Arcelor SA"],
            "Tires": ["Michelin", "Bridgestone Corp.", "Continental AG", "Goodyear Tire & Rubber Co", "Sumitomo Rubber Industries"],
            "Energy & water": ["GazProm", "Duke Energy", "Petrobras", "Petro China", "RosNeft", "BP plc", "Exxon Mobil Corp.", "Total SA", 'Royal Dutch Shell PLC', "Chevron Corp."],
            "Automotive & transport": ["General Motors Corp.", "BMW", "ABB", "Daimler AG", "Ford Motor Co.", "Toyota Motor Corp.", "Volkswagen AG"],
            "Wood & paper products": ["International Paper Corp.", "Georgia Pacific", "Weyerhaeuser Co.", "Stora Enso Oyj", "Kimberly-Clark Corp."],
            "Computer services & software": ["IBM", "Microsoft Corp.", "EDS Corp.", "AuFeminin.com", "Amazon.com Inc.", "Apple Inc"],
#             "Dirty Dozen": ["GazProm", "Petrobras", "RosNeft", "Petro China", "Duke Energy"]             
#             "Custom": [""]
}
companies_dc = {}
for cat, companies in categories.items():
    for c in companies:
        companies_dc[c] = cat
cats = list(categories.keys())
final_df['category'] = final_df.apply(lambda row: companies_dc[row['company']], axis=1)
import matplotlib.pyplot as plt
from matplotlib import colors
def get_cmap(n, name='hsv'):
    '''Returns a function that maps each index in 0, 1, ..., n-1 to a distinct 
    RGB color; the keyword argument name must be a standard mpl colormap name.'''
    return plt.cm.get_cmap(name, n)
cmap = get_cmap(len(cats))
#http://www.climatechangenews.com/2015/05/21/the-dirty-dozen-the-fossil-fuel-industrys-polluting-league-table/
dirty_dozen = ["GazProm", "Petrobras", "RosNeft", "Petro China", "Glencore International AG", 'Royal Dutch Shell PLC', "Exxon Mobil Corp.", "BP plc", "Chevron Corp."]

def highlight_categories(row):
    return [f'background-color: {colors.rgb2hex(cmap(cats.index(row["category"])))}' for i in row]

def highlight_dirty_dozen(row):
    if row["company"] in dirty_dozen:
        return ['background-color: red' for i in row]
    else:
        return ["" for i in row]


### Order by absolute number of negatives/positives.  Highlight sectors

In [23]:
final_df['mark'] = final_df['neg_c']/final_df['pos_c']
final_df = final_df.sort_values(['mark'], ascending=[0])
final_df.style.apply(highlight_categories, axis=1)

,company,neg_c,neg_p,neg_url,pos_c,pos_p,pos_url,category,mark
0,Intel,5,4.8999,Clickme,15,13.202,Clickme,Custom,0.333333
